In [1]:
%matplotlib inline
import warnings; warnings.simplefilter('ignore')

import fnmatch
import os
import pandas as pd
import geopandas as gpd

from datacube_stats.statistics import GeoMedian
from datacube.helpers import ga_pq_fuser
from datacube.storage import masking
from datacube.helpers import write_geotiff
import xarray as xr

#get the DEA version of the plotting functions
import sys
sys.path.append(os.path.abspath('/g/data/r78/DPIPWE_lm/dea-notebooks/10_Scripts'))
sys.path.append(os.path.abspath('/g/data/r78/DPIPWE_lm/datacube-2nd-order-stats'))
import DEAPlotting
import DEADataHandling


import datacube
dc = datacube.Datacube(app='stats_2nd_testing')

In [2]:
#x, y = (1385000.0, 1390000.0), (-4570000.0, -4575000.0)
sensors = ['ls8'] #take or remove as needed
deriv = 'nbart'
#product = 'nbart' #
time = ('2017-10-01', '2017-12-31')
resolution = (-25,25)
bands = ['red', 'green', 'blue', 'nir', 'swir1', 'swir2']
#epoch = ('2016', '2017') # time query for datacube function can be just years

In [3]:
query = {'x': (1300000.0, 1350000.0),
         'y': (-4700000.0, -4750000.0),
         'time': time,
         'resolution': resolution,
         'crs': 'EPSG:3577'}

In [4]:
dsma = DEADataHandling.load_clearlandsat(dc=dc, query=query,
                                          #product=product,
                                          masked_prop=0,
                                          sensors = sensors,
                                          bands_of_interest = bands,
                                          mask_pixel_quality=True,
                                          ls7_slc_off=True)
'''
dsma = dc.load(product='ls8_nbart_albers', group_by = 'solar_day', **query)
'''

Loading ls8 pixel quality
    Loading 11 filtered ls8 timesteps
Combining and sorting ls8 data
    Replacing invalid -999 values with NaN (data will be coerced to float64)


"\ndsma = dc.load(product='ls8_nbart_albers', group_by = 'solar_day', **query)\n"

In [1]:
# Load PQ data for same query used to load Landsat data
pq_dsma = dc.load(product = 'ls8_pq_albers',
                group_by = 'solar_day',
                fuse_func=ga_pq_fuser,
                **query)

# Use PQ to create mask that is True for pixels that are not affected by clouds, cloud shadow or saturation
good_quality = masking.make_mask(pq_dsma.pixelquality,
                                 cloud_acca='no_cloud',
                                 cloud_fmask='no_cloud',
                                 cloud_shadow_acca='no_cloud_shadow',
                                 cloud_shadow_fmask='no_cloud_shadow',
                                 blue_saturated=False,
                                 green_saturated=False,
                                 red_saturated=False,
                                 nir_saturated=False,
                                 swir1_saturated=False,
                                 swir2_saturated=False,
                                 contiguous=True)

NameError: name 'dc' is not defined

In [48]:
# Remove -999 nodata values prior to analysing or plotting Landsat imagery by setting all nodata values to `NaN`
dsma = masking.mask_invalid_data(dsma)

# Apply the mask to preserve only the good data
dsma = dsma.where(good_quality)

In [5]:
dsma
# Proably need to drop the 'data_perc' band (or load data via different method?)

<xarray.Dataset>
Dimensions:    (time: 11, x: 2000, y: 2000)
Coordinates:
  * y          (y) float64 -4.7e+06 -4.7e+06 -4.7e+06 ... -4.75e+06 -4.75e+06
  * x          (x) float64 1.3e+06 1.3e+06 1.3e+06 ... 1.35e+06 1.35e+06
  * time       (time) datetime64[ns] 2017-10-06T23:52:33 ... 2017-12-25T23:52:28
Data variables:
    red        (time, y, x) float64 455.0 529.0 519.0 ... 243.0 247.0 257.0
    green      (time, y, x) float64 410.0 472.0 494.0 ... 678.0 687.0 690.0
    blue       (time, y, x) float64 272.0 292.0 290.0 ... 694.0 702.0 704.0
    nir        (time, y, x) float64 1.932e+03 2.07e+03 2.047e+03 ... 229.0 238.0
    swir1      (time, y, x) float64 1.441e+03 1.957e+03 2.06e+03 ... 182.0 179.0
    swir2      (time, y, x) float64 748.0 1.08e+03 1.246e+03 ... 154.0 157.0
    data_perc  (time, y, x) float64 0.6239 0.6239 0.6239 ... 0.8521 0.8521
Attributes:
    crs:      EPSG:3577

In [6]:
dsma = dsma.drop('data_perc')

In [7]:
dsma

<xarray.Dataset>
Dimensions:  (time: 11, x: 2000, y: 2000)
Coordinates:
  * y        (y) float64 -4.7e+06 -4.7e+06 -4.7e+06 ... -4.75e+06 -4.75e+06
  * x        (x) float64 1.3e+06 1.3e+06 1.3e+06 ... 1.35e+06 1.35e+06 1.35e+06
  * time     (time) datetime64[ns] 2017-10-06T23:52:33 ... 2017-12-25T23:52:28
Data variables:
    red      (time, y, x) float64 455.0 529.0 519.0 438.0 ... 243.0 247.0 257.0
    green    (time, y, x) float64 410.0 472.0 494.0 449.0 ... 678.0 687.0 690.0
    blue     (time, y, x) float64 272.0 292.0 290.0 262.0 ... 694.0 702.0 704.0
    nir      (time, y, x) float64 1.932e+03 2.07e+03 2.047e+03 ... 229.0 238.0
    swir1    (time, y, x) float64 1.441e+03 1.957e+03 2.06e+03 ... 182.0 179.0
    swir2    (time, y, x) float64 748.0 1.08e+03 1.246e+03 ... 151.0 154.0 157.0
Attributes:
    crs:      EPSG:3577

In [8]:
from model import SMAD, BCMAD, TernaryMAD
#import datacube_stats

In [9]:
dsma_tmad = TernaryMAD().compute(dsma)

In [10]:
#dsma_tmad
dsma_tmad

<xarray.Dataset>
Dimensions:  (x: 2000, y: 2000)
Dimensions without coordinates: x, y
Data variables:
    sdev     (y, x) float32 0.00060243276 0.00057326315 ... 0.014132628
    edev     (y, x) float32 0.018163987 0.023457155 ... 0.024160132 0.024775576
    bcdev    (y, x) float32 0.03227721 0.03437012 ... 0.2002736 0.20013557

In [10]:
dsma_tmad.bcdev

<xarray.DataArray 'bcdev' (y: 1000, x: 1000)>
array([[0.111488, 0.112365, 0.129521, ..., 0.118699, 0.179835, 0.107692],
       [0.039324, 0.097063, 0.070703, ..., 0.20067 , 0.114142, 0.176149],
       [0.      , 0.200132, 0.113289, ..., 0.145098, 0.188263, 0.172206],
       ...,
       [0.049541, 0.051778, 0.036684, ..., 0.252089, 0.178835, 0.084949],
       [0.046041, 0.040124, 0.029713, ..., 0.15886 , 0.061667, 0.110157],
       [0.038469, 0.039168, 0.039816, ..., 0.352061, 0.139128, 0.174838]],
      dtype=float32)
Dimensions without coordinates: y, x

In [11]:
#ds=xr.Dataset({'bcmad': (['y','x'], dsma_smad.bcdev)}, coords={'x': dsma.x, 'y':dsma.y}, attrs=dsma.attrs)
ds=xr.Dataset({'smad': (['y','x'], dsma_tmad.sdev), 
               'emad': (['y','x'], dsma_tmad.edev), 
               'bcmad': (['y','x'], dsma_tmad.bcdev)},
                coords={'x': dsma.x, 'y':dsma.y}, attrs=dsma.attrs)

In [12]:
ds

<xarray.Dataset>
Dimensions:  (x: 2000, y: 2000)
Coordinates:
  * x        (x) float64 1.3e+06 1.3e+06 1.3e+06 ... 1.35e+06 1.35e+06 1.35e+06
  * y        (y) float64 -4.7e+06 -4.7e+06 -4.7e+06 ... -4.75e+06 -4.75e+06
Data variables:
    smad     (y, x) float32 0.00060243276 0.00057326315 ... 0.014132628
    emad     (y, x) float32 0.018163987 0.023457155 ... 0.024160132 0.024775576
    bcmad    (y, x) float32 0.03227721 0.03437012 ... 0.2002736 0.20013557
Attributes:
    crs:      EPSG:3577

In [13]:
ds

<xarray.Dataset>
Dimensions:  (x: 2000, y: 2000)
Coordinates:
  * x        (x) float64 1.3e+06 1.3e+06 1.3e+06 ... 1.35e+06 1.35e+06 1.35e+06
  * y        (y) float64 -4.7e+06 -4.7e+06 -4.7e+06 ... -4.75e+06 -4.75e+06
Data variables:
    smad     (y, x) float32 0.0055516004 0.0015026127 ... 0.007496543
    emad     (y, x) float32 0.085933276 0.04128909 ... 0.014507783 0.013779247
    bcmad    (y, x) float32 0.1285931 0.07117009 ... 0.12122768 0.11289027
Attributes:
    crs:      EPSG:3577

In [22]:
#dsma_smad.attrs['crs'] = 'EPSG:3577'
#dsma_smad.attrs['affine'] = dsma.affine

In [13]:
#datacube.storage.storage.write_dataset_to_netcdf(dsma_smad, '/g/data/r78/DPIPWE_LM/output_data/ls8_smad_test.nc')
datacube.helpers.write_geotiff(filename='/g/data/r78/DPIPWE_lm/output_data/ls8_all_TMAD_2016_chip.tif', dataset=ds)
#DEADataHandling.dataset_to_geotiff('dsma_smad_netcdf_test.nc', dsma_smad)